# Kwansi Example Implementation

This notebook demonstrates the use of Kwansi, a wrapper for DSPy that makes its optimizers easier to use. We'll walk through the example implementation provided in `example_implementation.py`.

## Introduction

Kwansi is a wrapper for [DSPy](https://dspy-docs.vercel.app), a framework for programming - not prompting - language models. DSPy is a powerful, modular tool that assists the way we work with language models (LMs) in complex systems.

Kwansi builds upon DSPy's capabilities with a focus on making making its optimizers easier to use - especially if you are using LLM-based evaluators. You can find more information about Kwansi and its implemented functions here: https://github.com/lordamp/kwansi

## Example

For this example, assume we have written a simple prompt for creating tweets based on a given topic and details:

```
Craft interesting tweets based on given topics and details. The tweets should be within the 280-character limit. Use a tone to suit the topic. Don't use hashtags.
```


We want to optimize this prompt.

## Setup

First, make sure you have all required packages (including the kwansi library) installed by running

`pip install -r requirements.txt`.

Then, set up your OpenAI API key in a `.env` file:
```
OPENAI_API_KEY=<your-key>
```

## Importing Libraries and Loading Environment Variables

First, let's import the necessary libraries and load the environment variables.

In [1]:
import dspy
import os
from dotenv import load_dotenv
import json

# Import the functions from the kwansi package
from kwansi.data_preparation import prepare_examples
from kwansi.evaluation import create_evaluator
from kwansi.optimizer_handling import run_optimizer, save_optimized_model
from kwansi.task_creation import create_task
from kwansi.testing import test_model

/home/lordampersand/Documents/6-Programming/2024-09-39_kwansi_example/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Environment Variables

Now, let's load the environment variables, so our LLM API key is loaded.

In [2]:
load_dotenv()

True

## Load Components

Next, we load the components we'll need for our task.

In [3]:
from components.task import TweetCreatorSignature
from components.assessors import Assess_Interestingness, Assess_StyleAppropriateness
from components.metrics import length_metric, hashtag_count_metric
from components.custom_combiner import custom_combine

Let's check out what we have here.

### Task

In [4]:
print(TweetCreatorSignature)

TweetCreatorSignature(topic, details -> tweet
    instructions="Craft interesting tweets based on given topics and details. The tweets should be within the 280-character limit. Use a tone to suit the topic. Don't use hashtags."
    topic = Field(annotation=str required=True json_schema_extra={'desc': 'the main subject of the tweet', '__dspy_field_type': 'input', 'prefix': 'Topic:'})
    details = Field(annotation=str required=True json_schema_extra={'desc': 'additional information or context for the tweet', '__dspy_field_type': 'input', 'prefix': 'Details:'})
    tweet = Field(annotation=str required=True json_schema_extra={'desc': 'a tweet', '__dspy_field_type': 'output', 'prefix': 'Tweet:'})
)


The `TweetCreatorSignature` is a "Signature" for our "Task", a custom DSPy class that contains the prompt we're trying to optimize, its input fields, and output field. It has the elements:

- `instructions`: The instructions for the task (the actual prompt)
- `topic` and `details`: Input fields the prompt expects
- `tweet`: The output field the prompt produces (a tweet)

This is an unoptimized version of the prompt, which is also why it's called the "Student" in the code.

### Assessors

Next, we have our assessors:

In [5]:
print(Assess_Interestingness)
print("-"*100)
print(Assess_StyleAppropriateness)

Assess_Interestingness(tweet -> score
    instructions='Assess how interesting and engaging the tweet is on a scale of 0 (very uninteresting) to 10 (highly interesting).'
    tweet = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Tweet:', 'desc': '${tweet}'})
    score = Field(annotation=str required=True json_schema_extra={'desc': "A score between 0 and 10 in the format 'Score: X'", '__dspy_field_type': 'output', 'prefix': 'Score:'})
)
----------------------------------------------------------------------------------------------------
Assess_StyleAppropriateness(tweet, topic -> score
    instructions='Assess if the style of the tweet is appropriate for the topic and platform on a scale of 0 (very inappropriate) to 10 (perfectly appropriate).'
    tweet = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Tweet:', 'desc': '${tweet}'})
    topic = Field(annotation=str required=True json_schema_ex

These are again two DSPy Signatures, i.e. prompts. They have a special role: They evaluate the quality of the task we are trying to optimize.  

- **Assess_Interestingness**: How interesting is the tweet?
- **Assess_StyleAppropriateness**: How appropriate is the style of the tweet?

Both assessors have an instructions field, as well as input fields the prompt expects and an output field that the prompt produces. The `Assess_Interestingness` assessor expects a `tweet` field as input, and produces a `score` field as output. This score will be between 0 and 10. The `Assess_StyleAppropriateness` assessor expects two fields as input: a `tweet` and a `topic`, and also produces a `score`.

### Metrics

Next, we have two metrics:

In [6]:
import inspect

print(inspect.getsource(length_metric))
print("-"*100)
print(inspect.getsource(hashtag_count_metric))

def length_metric(example, pred):
    """
    Check if the tweet is below the 280 character limit.
    Returns 1 if the tweet is below the limit, 0 if it is not.
    """
    return 1 if len(pred.tweet) <= 280 else 0

----------------------------------------------------------------------------------------------------
def hashtag_count_metric(example, pred):
    """
    Count hashtags in the tweet.
    Returns 1 if there are no hashtags, 0 if there are any hashtags.
    """
    tweet = pred.tweet
    hashtag_count = tweet.count('#')
    
    return 1 if hashtag_count == 0 else 0



These are two simple metrics. `length_metric` checks if the tweet is below the 280 character limit, and `hashtag_count_metric` checks if there are any hashtags in the tweet. They both return a binary score - 1 if the condition is met (equal to or below 280 characters, no hashtags), 0 if it is not (above 280 characters, hashtags present).

This means our task gets evaluated and hence optimized for:
- Being interesting
- Being appropriate
- Being below the character limit
- Having no hashtags

### Custom Combiner

Finally, we have a another function:

In [7]:
print(inspect.getsource(custom_combine))

def custom_combine(scores):
    # Weighted combination of first two scores, multiplied by the other two
    weights = [0.7, 0.3]
    weighted_sum = sum(scores[i][1] * weights[i] for i in range(2))
    return weighted_sum * scores[2][1] * scores[3][1]



This function takes the weighted sum of the first two scores, and multiplies it by the other two scores. This is a custom combiner, which is a function that takes the scores from the assessors and metrics and combines them in a custom way. We need this because we have four scores, and can only have one KPI to optimize our task for.

As you will see, it's not necessary to have a custom combiner. You can also define it to use a simple additive method, or multiplicative method via a keyword.

Next, we'll take a look at the data we'll use to optimize our task.

## Load and Prepare Data

In our case, we load the data directly from a JSON file. You might want to load it from a different format, or even a database, but in the end, we'll need JSON data to pass to DSPy.

In [8]:
# Load the data - make sure to transform to JSON format
with open('data/example_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

What does the data look like?

In [9]:
print("Keys in the data dictionary:")
print(list(data.keys()))
print("-"*100)
print("First example from 'tweet_instructions':")
print(json.dumps(data['tweet_instructions'][0], indent=2))
print("-"*100)
print("Number of examples in 'tweet_instructions':")
print(len(data['tweet_instructions']))



Keys in the data dictionary:
['tweet_instructions']
----------------------------------------------------------------------------------------------------
First example from 'tweet_instructions':
{
  "id": 1,
  "topic": "Product launch",
  "details": "software suite for a very user friendly ERP system, Danish company called ErstHagen, by January 2025",
  "author": "Emma Carsten"
}
----------------------------------------------------------------------------------------------------
Number of examples in 'tweet_instructions':
100


The data is a dictionary with a key "tweet_instructions", which contains a list of dictionaries. Each dictionary represents an example, with keys "topic", "details" and "author". We have 100 examples in total.

We can now use this data to define the input fields instructions for the task. DSPy expects the data to be in a data format called "Example", which is a list of dictionaries. In our case, each dictionary represents an example, with keys "topic" and "details". We don't need the "author" field for the task, so we can exclude it.

We can now prepare the examples, but will only take 50 of them for efficiency.

In [10]:
input_fields = {
    'data_key': 'tweet_instructions',  # the key in the JSON file that contains the data
    'fields': ['topic', 'details']  # the fields in the JSON file that are the input to the task
}

tweet_examples = prepare_examples(data, input_fields, n_samples=50)

How do they look like?

In [11]:
print(f"Number of examples prepared: {len(tweet_examples)}")
print("Sample example:")
print(tweet_examples[0])

Number of examples prepared: 50
Sample example:
Example({'topic': 'Educational content', 'details': '5 tips for improving cybersecurity for small businesses, focus on employee training'}) (input_keys={'topic', 'details'})


## Create the Task

Next, we create the task we want to optimize. For this, we use our task signature from `components/task.py` we had loaded earlier.

Our task is called `TweetCreator`, and the DSPy module we're using is `ChainOfThought`, because we want our LLM to think through the task before responding. Alternatively, we could have used `Predict`, which is a simpler response instruction.


In [12]:
TweetCreator = create_task(TweetCreatorSignature, 'ChainOfThought')
tweet_creator = TweetCreator()

print("Task created:")
print(tweet_creator)

Task created:
executor = Predict(StringSignature(topic, details -> rationale, tweet
    instructions="Craft interesting tweets based on given topics and details. The tweets should be within the 280-character limit. Use a tone to suit the topic. Don't use hashtags."
    topic = Field(annotation=str required=True json_schema_extra={'desc': 'the main subject of the tweet', '__dspy_field_type': 'input', 'prefix': 'Topic:'})
    details = Field(annotation=str required=True json_schema_extra={'desc': 'additional information or context for the tweet', '__dspy_field_type': 'input', 'prefix': 'Details:'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the tweet}. We ...', '__dspy_field_type': 'output'})
    tweet = Field(annotation=str required=True json_schema_extra={'desc': 'a tweet', '__dspy_field_type': 'output', 'prefix': 'Tweet:'})
))


## Define the Evaluator

We have the task, we have the data, now the last ingredient we need is a our evaluator.

In [13]:
tweet_evaluator = create_evaluator(
    assessors=[
        ('Interestingness', Assess_Interestingness, {'tweet': 'tweet'}, (0, 10)),
        ('Style_Appropriateness', Assess_StyleAppropriateness, {'tweet': 'tweet', 'topic': 'topic'}, (0, 10))
    ],
    additional_metrics=[
        ('Length_Check', length_metric),
        ('Hashtag_Count', hashtag_count_metric)
    ],
    combine_method="multiplicative",
    threshold=0.25
)

Let's take this function all apart.

First, we define the evaluator `tweet_evaluator` with the `create_evaluator` function. We pass the following arguments:

1. `assessors`: Our list of assessor, which contain the name of the assessor (`'Interestingness'` and `'Style_Appropriateness'`), the assessor itself (`Assess_Interestingness`, `Assess_StyleAppropriateness`), the input fields the assessor expects (`tweet` for both, `topic` for `Assess_StyleAppropriateness`), and the range of scores the assessor can produce (0-10) .

2. `additional_metrics`: Our list of metrics, which call `Length_Check` (referring to the `length_metric`) and `Hashtag_Count` (referring to the `hashtag_count_metric`).

3. `combine_method`: The method to combine the assessor scores and additional metrics into a single score. We use `multiplicative` here, but could also use `"additive"` or our function `custom_combine` here.

4. `threshold`: The minimum score for the evaluator to be considered passing. While optimizing, if the score of one of our examples is below that value, it will be abandoned. The value depends on the range of scores our assessors can produce. Here, our value is 0.25, which is based on the fact that our two assessors are multiplied. If they both scored 0.5, the evaluator would score 0.25. Note that we ignore the metrics here, as in a multiplicative method, they are either 1 (pass) or 0 (fail).

## Define the Language Model

Everything we have defined so far is independent of the language model we use. We can now choose one and define it. We'll use `gpt-4o-mini` for this example, but you can choose any other model that is compatible with DSPy. Note that kwansi has not yet been tested with other LLMs than OpenAI's. 

In [14]:
# Define the language model DSPy will use
dspy.settings.configure(lm=dspy.LM(
    model='gpt-4o-mini',
    api_key=os.environ['OPENAI_API_KEY'],
    max_tokens=1024
))

## Run the Optimizer

Now, let's run the optimizer to improve our task. We pass the following arguments:

1. `optimizer_type`: The type of optimizer to use. DSPy offers a variety of optimizers (including finetuning); in kwansi, we have implemented a few of them:
    - `BootstrapFewShot`: Based on BootstrapFewShot optimizer from DSPy, but uses LLM-based evaluators.
    - `BootstrapFewShotWithRandomSearch`: Based on BootstrapFewShot, this optimizer uses random search to further optimize the task.
    - `COPRO`: Optimizes only the prompt without few-shot examples.
    - `MIPROv2`: Optimizes both the prompt and the few-shot examples.
    - `MIPROv2ZeroShot`: Like MIPROv2, but optimizes only the prompt without few-shot examples.

    Please check out the DSPy documentation (["Which optimizer should I use?"](https://dspy-docs.vercel.app/docs/building-blocks/optimizers#which-optimizer-should-i-use)) for more information on the optimizers and the number of examples they need.

2. `evaluator`: The evaluator to optimize for. We use `tweet_evaluator` we defined earlier.

3. `student`: Our task, `tweet_creator`.

4. `trainset`: Our training data, `tweet_examples`. Note that the evaulation set is automatically extracted from the training data.

As we have 50 examples, `BootstrapFewShotWithRandomSearch` is a good choice.

In [15]:
# Run the optimizer (i.e. the process of optimizing the task)
optimized_tweet_creator, optimizer_type = run_optimizer(
    optimizer_type='BootstrapFewShotWithRandomSearch',
    evaluator=tweet_evaluator,
    student=tweet_creator,
    trainset=tweet_examples,
)

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 16 candidate sets.


Average Metric: 44.835999999999984 / 50  (89.7): 100%|██████████| 50/50 [00:05<00:00,  8.80it/s]


New best score: 89.67 for seed -3
Scores so far: [89.67]
Best score so far: 89.67


Average Metric: 44.835999999999984 / 50  (89.7): 100%|██████████| 50/50 [00:00<00:00, 3377.00it/s]


Scores so far: [89.67, 89.67]
Best score so far: 89.67


...

Scores so far: [89.67, 89.67, 90.39, 90.61, 90.24, 90.69, 90.55, 90.33, 90.4, 90.55, 90.47, 90.4, 90.46, 90.39, 89.96, 90.39, 90.24, 89.44]
Best score so far: 90.69


  4%|▍         | 2/50 [00:00<00:00, 578.84it/s]


Bootstrapped 2 full traces after 3 examples in round 0.


Average Metric: 45.23599999999998 / 50  (90.5): 100%|██████████| 50/50 [00:04<00:00, 10.22it/s] 

Scores so far: [89.67, 89.67, 90.39, 90.61, 90.24, 90.69, 90.55, 90.33, 90.4, 90.55, 90.47, 90.4, 90.46, 90.39, 89.96, 90.39, 90.24, 89.44, 90.47]
Best score so far: 90.69
19 candidate programs found.


Depending on the optimizer you choose, you will see different iterations of the optimizer and the scores. 

## Save the Optimized Model

Our model is now optimized. We can save it for future use (don't forget that, or you'll have to optimize it again). We save it in the `output` folder, and name it `tweet_creator`. The saving function automatically inludes the optimizer type and datetime in the filename.

In [16]:
save_optimized_model(optimized_tweet_creator, optimizer_type, folder='output', name='tweet_creator')

[('executor', Predict(StringSignature(topic, details -> rationale, tweet
    instructions="Craft interesting tweets based on given topics and details. The tweets should be within the 280-character limit. Use a tone to suit the topic. Don't use hashtags."
    topic = Field(annotation=str required=True json_schema_extra={'desc': 'the main subject of the tweet', '__dspy_field_type': 'input', 'prefix': 'Topic:'})
    details = Field(annotation=str required=True json_schema_extra={'desc': 'additional information or context for the tweet', '__dspy_field_type': 'input', 'prefix': 'Details:'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the tweet}. We ...', '__dspy_field_type': 'output'})
    tweet = Field(annotation=str required=True json_schema_extra={'desc': 'a tweet', '__dspy_field_type': 'output', 'prefix': 'Tweet:'})
)))]
Optimized model saved to output/2024-09-29_16-41-45_tweet_crea

## Test the Optimized Model

This step is optional, but we want to make sure our model is working as expected. For this, we can use the `test_model` function. We pass the following arguments:

1. `model`: Our optimized model, `optimized_tweet_creator`.
2. `test_data`: Our training data, `tweet_examples`.
3. `n_tests`: The number of tests to run. We use 3.
4. `input_fields`: The input fields the model expects. We use `topic` and `details`.
5. `output_field`: The output field the model produces. We use `tweet`.
6. `evaluator`: The evaluator to use. We use `tweet_evaluator` we defined earlier.
7. `verbose`: Whether to print a long or short version of the results. We want to see the details, so we set `verbose=True`.


In [17]:
# Test the Optimized Program (use verbose=True for more details)
print("Short Test Output:")
test_model(
    model=optimized_tweet_creator,
    test_data=tweet_examples,
    n_tests=1,
    input_fields=['topic', 'details'],
    output_field='tweet',
    evaluator=tweet_evaluator,
    verbose=True
)

Short Test Output:
Test 1:
Topic: Educational content
Details: 5 tips for improving cybersecurity for small businesses, focus on employee training
Generated tweet: Small businesses, listen up! Here are 5 tips to boost your cybersecurity: 1) Conduct regular training sessions, 2) Simulate phishing attacks, 3) Encourage strong passwords, 4) Keep software updated, 5) Foster a culture of security awareness. Protect your business today!
Evaluator scores:
  Interestingness: 0.8000
  Style_Appropriateness: 0.9000
  Length_Check: 1
  Hashtag_Count: 1
  Total_Score: 0.7200
--------------------------------------------------


## Iterate!

It's likely that your first iteration won't be perfect. That's why you should iterate.

### Tweak the Evaluator

In most cases, this will mean tweaking your evaluator, because you haven't defined it well enough to capture what "good" looks like - very likely it won't be what you initially thought it is. Check out the example output to see what the optimized evaluator considers a good output. For example, in the `interestingness` signature, you might want to add instructions explanations on what content and style makes a tweet interesting and what doesn't. 

Maybe it's another metric you need - for example, a counter for emojis to boost or reduce their number. Or you find some edge cases that you hadn't considered in the first place, which you can add to the evaluator as well.

### Add More/Different Data

Maybe you also find out that you will need more context for the task to work, so you will have to add additional data and input fields. For example, `appropriateness` might be different for various target groups, so including the age group or interests of the author might be necessary - if you have that data!

### Tweak the Task

In some cases, you might also want to tweak your task, especially if you see that your optimizer can't get above the threshold you set. For example, in writing this example, it became clear that the task needed an instruction on avoiding hashtags, otherwise the task would always produce them and get a 0 score in the `Hashtag_Count` metric, leading to an evaluator score of 0. If you have used COPRO or MIPROv2 to optimize your prompt, you can also take this optimized instruction and add it as the new student for the next iteration.

## Conclusion

If you want to learn more about DSPy, you can find the documentation here: https://dspy-docs.vercel.app/

If you have any questions or suggestions, please feel free to open an issue on Github: https://github.com/lordamp/kwansi/issues. 

Thanks for trying out Kwansi!

